In [158]:
# Data manipulation and visualization
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Clustering and distance metrics
from sklearn.cluster import KMeans
from scipy.spatial import distance

# Preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Model evaluation
from sklearn.metrics import silhouette_score

# Pipeline and dimensionality reduction
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

# Elbow method for KMeans
from yellowbrick.cluster import KElbowVisualizer
pd.set_option('display.max_columns', None)

In [159]:
olist = pd.read_csv("Olist - E-Commerce Customer Data.csv").drop(columns='Unnamed: 0')
olist

,order_id,order_item_id,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_id,seller_city,seller_state,seller_zip_code_prefix,payment_type,payment_sequential,payment_installments,installments_price,price,freight_value,payment_value,shipping_limit_date,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,shipping_duration,day_of_purchase,month_of_purchase,year_of_purchase,month/year_of_purchase,order_status,order_unique_id
0,00010242fe8c5a6d1ba2dd792cb16214,1,3ce436f183e68e07877b285a838db11a,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,48436dade18ac8b2bce089ec2a041202,volta redonda,SP,27277,credit_card,1,2,36.0,58.9,13.29,72.19,2017-09-19 09:45:35,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,7 days 14:44:46,Wednesday,September,2017,September-2017,delivered,00010242fe8c5a6d1ba2dd792cb16214-1
1,130898c0987d1801452a8ed92a670612,1,e6eecc5a77de221464d1c4eaff0a9b64,0fb8e3eab2d3e79d92bb3fffbb97f188,75800,jatai,GO,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,48436dade18ac8b2bce089ec2a041202,volta redonda,SP,27277,boleto,1,1,74.0,55.9,17.96,73.86,2017-07-05 02:44:11,2017-06-28 11:52:20,2017-06-29 02:44:11,2017-07-05 12:00:33,2017-07-13 20:39:29,2017-07-26 00:00:00,15 days 08:47:09,Wednesday,June,2017,June-2017,delivered,130898c0987d1801452a8ed92a670612-1
2,532ed5e14e24ae1f0d735b91524b98b9,1,4ef55bf80f711b372afebcb7c715344a,3419052c8c6b45daf79c1e426f9e9bcb,30720,belo horizonte,MG,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,48436dade18ac8b2bce089ec2a041202,volta redonda,SP,27277,credit_card,1,2,41.5,64.9,18.33,83.23,2018-05-23 10:56:25,2018-05-18 10:25:53,2018-05-18 12:31:43,2018-05-23 14:05:00,2018-06-04 18:34:26,2018-06-07 00:00:00,17 days 08:08:33,Friday,May,2018,May-2018,delivered,532ed5e14e24ae1f0d735b91524b98b9-1
3,6f8c31653edb8c83e1a739408b5ff750,1,30407a72ad8b3f4df4d15369126b20c9,e7c828d22c0682c1565252deefbe334d,83070,sao jose dos pinhais,PR,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,48436dade18ac8b2bce089ec2a041202,volta redonda,SP,27277,credit_card,1,3,25.0,58.9,16.17,75.07,2017-08-07 18:55:08,2017-08-01 18:38:42,2017-08-01 18:55:08,2017-08-02 19:07:36,2017-08-09 21:26:33,2017-08-25 00:00:00,8 days 02:47:51,Tuesday,August,2017,August-2017,delivered,6f8c31653edb8c83e1a739408b5ff750-1
4,7d19f4ef4d04461989632411b7e588b9,1,91a792fef70ecd8cc69d3c7feb3d12da,0bb98ba72dcc08e95f9d8cc434e9a2cc,36400,conselheiro lafaiete,MG,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,48436dade18ac8b2bce089ec2a041202,volta redonda,SP,27277,credit_card,1,4,18.0,58.9,13.29,72.19,2017-08-16 22:05:11,2017-08-10 21:48:40,2017-08-10 22:05:11,2017-08-11 19:43:07,2017-08-24 20:04:21,2017-09-01 00:00:00,13 days 22:15:41,Thursday,August,2017,August-2017,delivered,7d19f4ef4d04461989632411b7e588b9-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113385,fdaf98feac227eb978d8f33f36e0231d,1,81ae83ce1a10af9f93fa3512bd9713b1,3d50a44231c2a153219cef76ee3e445d,22793,rio de janeiro,RJ,f739e9151702508b18f796c53005e5e9,moveis_decoracao,44.0,533.0,2.0,1000.0,69.0,11.0,11.0,d1aa1ec0839dcab73a6161130eb1f94a,curitiba,PR,80220,boleto,1,1,136.0,119.9,16.14,136.04,2017-04-27 05:10:28,2017-04-19 10:20:51,2017-04-21 05:10:28,2017-05-04 15:06:24,2017-05-24 10:52:43,2017-05-22 00:00:00,35 days 00:31:52,Wednesday,April,2017,April-2017,delivered,fdaf98feac227eb978d8f33f36e0231d-1
113386,fe68b52db13993f58175f

## Redução dos dados

In [160]:
print(olist.shape)
olist.columns

(113390, 40)


Index(['order_id', 'order_item_id', 'customer_id', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'seller_id', 'seller_city', 'seller_state', 'seller_zip_code_prefix',
       'payment_type', 'payment_sequential', 'payment_installments',
       'installments_price', 'price', 'freight_value', 'payment_value',
       'shipping_limit_date', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'shipping_duration', 'day_of_purchase',
       'month_of_purchase', 'year_of_purchase', 'month/year_of_purchase',
       'order_status', 'order_unique_id'],
      dtype='object')

In [161]:
num_col = [ # User data
           'order_item_id', 'customer_zip_code_prefix', 'seller_zip_code_prefix'
           # Product data
           'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 
           'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm',
           # Payment data
           'payment_sequential', 'payment_installments', 'installments_price', 'price', 'freight_value', 'payment_value']

In [162]:
dataset, amostra = train_test_split(olist, train_size = .10)
dataset = pd.DataFrame(dataset)

dataset_KMeans = dataset[['payment_sequential', 'payment_installments', 'installments_price', 'price', 'freight_value', 'payment_value', 
                          'product_category_name', 'customer_state', 'seller_state']]
num_col = list(dataset_KMeans.select_dtypes('number').columns)
cat_col = list(dataset_KMeans.select_dtypes('object').columns)

print(dataset_KMeans.shape)
dataset_KMeans.head()

(11339, 9)


,payment_sequential,payment_installments,installments_price,price,freight_value,payment_value,product_category_name,customer_state,seller_state
19750,1,1,165.0,148.00,17.48,165.48,cool_stuff,RJ,PR
96368,1,1,36.0,19.99,16.11,36.10,esporte_lazer,GO,SP
88775,1,1,324.0,279.90,43.82,323.72,moveis_escritorio,SP,SP
99096,1,1,370.0,319.90,50.32,370.22,moveis_decoracao,SP,SP
111840,1,1,59.0,45.99,12.79,58.78,utilidades_domesticas,SP,SP


In [163]:
# Utilizamos OneHotEncoder para as variáveis categóricas e StandardScaler para as numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_col),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_col)  # Drop='first' para evitar multicolinearidade
    ])

X_preprocessed = preprocessor.fit_transform(dataset_KMeans)
X_preprocessed

array([[-0.13874568, -0.69981206,  0.48950049, ...,  0.        ,
         0.        ,  0.        ],
       [-0.13874568, -0.69981206, -0.34930305, ...,  0.        ,
         0.        ,  1.        ],
       [-0.13874568, -0.69981206,  1.52337461, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.13874568,  2.53859182, -0.4533407 , ...,  0.        ,
         0.        ,  1.        ],
       [-0.13874568,  0.73947855, -0.28557999, ...,  0.        ,
         0.        ,  0.        ],
       [-0.13874568,  0.3796559 ,  0.50738196, ...,  0.        ,
         0.        ,  0.        ]])

In [164]:
kmeans = KMeans(n_clusters=5, max_iter=600, algorithm = 'lloyd')
kmeans.fit(X_preprocessed)

KMeans(max_iter=600, n_clusters=5)

In [165]:
dataset['grupo'] = kmeans.labels_
#sns.pairplot(dataset, hue='grupo')

In [166]:
dataset.head()

,order_id,order_item_id,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_id,seller_city,seller_state,seller_zip_code_prefix,payment_type,payment_sequential,payment_installments,installments_price,price,freight_value,payment_value,shipping_limit_date,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,shipping_duration,day_of_purchase,month_of_purchase,year_of_purchase,month/year_of_purchase,order_status,order_unique_id,grupo
19750,1612d79764f7941116acc8be4239906b,1,91f9de3e080ef6be6aa8b5ae3cae1d4f,bd84ecd29495e3aa78cf6491ee2e91af,27523,resende,RJ,e59dd207c69d86e890febadc796d1078,cool_stuff,54.0,850.0,1.0,125.0,16.0,12.0,12.0,e9bc59e7b60fc3063eb2290deda4cced,maringa,PR,87083,boleto,1,1,165.0,148.00,17.48,165.48,2018-03-05 03:10:34,2018-02-24 14:59:24,2018-02-27 03:10:34,2018-03-14 01:11:23,2018-03-22 22:14:53,2018-03-22 00:00:00,26 days 07:15:29,Saturday,February,2018,February-2018,delivered,1612d79764f7941116acc8be4239906b-1,0
96368,53d515fbf0a0d3b35e47a45a398c551d,1,1bece94d7b280902f62747628ac0891b,9066d003674f53303ec2ac527e940a5d,75830,mineiros,GO,d370e908175e2f3b39bb7fd33fd4c530,esporte_lazer,60.0,972.0,1.0,400.0,20.0,20.0,20.0,751bdc4d83a466c7206cd42e8f426b03,ribeirao pires,SP,9405,boleto,1,1,36.0,19.99,16.11,36.10,2017-07-11 03:03:59,2017-07-04 15:54:10,2017-07-05 03:03:59,2017-07-05 11:46:38,2017-07-11 16:32:38,2017-08-01 00:00:00,7 days 00:38:28,Tuesday,July,2017,July-2017,delivered,53d515fbf0a0d3b35e47a45a398c551d-1,0
88775,4ef992d01b3b8db02c17acd3484e20de,1,607b751eb3decd6ce324f68150f2f1b9,a0cbc0fd1674542e2946647266a76b3d,14062,ribeirao preto,SP,38f82b1c10c441adf66cda952e0b6096,moveis_escritorio,52.0,2250.0,1.0,11950.0,58.0,21.0,56.0,17a053fcb14bd219540cbde0df490be0,mogi guacu,SP,13843,boleto,1,1,324.0,279.90,43.82,323.72,2018-05-04 03:30:31,2018-04-26 11:38:30,2018-04-27 03:30:39,2018-05-04 13:12:00,2018-05-10 20:47:48,2018-05-21 00:00:00,14 days 09:09:18,Thursday,April,2018,April-2018,delivered,4ef992d01b3b8db02c17acd3484e20de-1,4
99096,2c113cbb0811cf0b32479bd0bce3379b,1,078e99036000210bb0d5abb62fdeecee,ee0a502ce0f319eb6a2f8de3ed0188da,14406,franca,SP,0cbca89e0ecfd4fe92dff4f1c27a0487,moveis_decoracao,42.0,660.0,6.0,14950.0,40.0,65.0,40.0,f7496d659ca9fdaf323c0aae84176632,sao paulo,SP,4156,boleto,1,1,370.0,319.90,50.32,370.22,2018-08-22 04:24:16,2018-08-12 18:24:04,2018-08-14 04:24:16,2018-08-22 08:46:00,2018-08-27 17:21:49,2018-08-29 00:00:00,14 days 22:57:45,Sunday,August,2018,August-2018,delivered,2c113cbb0811cf0b32479bd0bce3379b-1,4
111840,4f420928ce3031d9833981e1bd5005a0,1,205e2ab340cb3c8bf457c76b98306c73,25063ca4218b4983df1daec133b143a2,15085,sao jose do rio preto,SP,d14c053e6b7bb2e2f819b1b4926f6159,utilidades_domesticas,41.0,1391.0,1.0,1307.0,41.0,19.0,28.0,a414555ce331b8c8aea4a9cb8395194d,sao paulo,SP,3638,voucher,1,1,59.0,45.99,12.79,58.78,2018-03-16 08:30:58,2018-03-10 15:31:45,2018-03-12 08:35:28,2018-03-12 21:21:54,2018-03-17 15:02:44,2018-03-28 00:00:00,6 days 23:30:59,Saturday,March,2018,March-2018,delivered,4f420928ce3031d9833981e1bd5005a0-1,0


In [167]:
dataset.grupo.value_counts()

grupo
0    8262
3    2322
4     638
1      65
2      52
Name: count, dtype: int64

In [168]:
dataset.groupby('grupo')['order_item_id'].mean()

grupo
0    1.153716
1    1.800000
2    1.211538
3    1.280362
4    1.442006
Name: order_item_id, dtype: float64

In [169]:
pd.crosstab(dataset['product_category_name'], dataset['grupo'])

grupo,0,1,2,3,4
product_category_name,,,,,
agro_industria_e_comercio,13,1,4,5,10
alimentos,46,0,0,4,0
alimentos_bebidas,30,0,0,2,0
artes,17,0,0,0,0
artes_e_artesanato,1,0,0,0,0
...,...,...,...,...,...
sinalizacao_e_seguranca,18,0,0,3,2
tablets_impressao_imagem,6,0,0,3,0
telefonia,398,0,0,40,12


In [170]:
"""# Elbow
def calcular_inercia(X, max_clusters=10):
    inercia = []
    for k in range(2, max_clusters):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(X)
        inercia.append(kmeans.inertia_)
    return inercia

# Silhouette
def calcular_silhouette(X, max_clusters=10):
    silhouette_scores = []
    for k in range(2, max_clusters):
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(X)
        score = silhouette_score(X, labels)
        silhouette_scores.append(score)
    return silhouette_scores

inercia = calcular_inercia(X_preprocessed)
silhouette_scores = calcular_silhouette(X_preprocessed)

# Plotar o gráfico
plt.figure(figsize=(14, 6))

# Gráfico do método Elbow
plt.subplot(1, 2, 1)
plt.plot(range(2, 10), inercia, marker='o', linestyle='--')
plt.title('Método Elbow para Determinação do Número de Clusters')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.grid()

# Gráfico da Pontuação de Silhouette
plt.subplot(1, 2, 2)
plt.plot(range(2, 10), silhouette_scores, marker='o', linestyle='--', color='orange')
plt.title('Pontuação de Silhouette para Diferentes Clusters')
plt.xlabel('Número de Clusters')
plt.ylabel('Silhouette Score')
plt.grid()

plt.tight_layout()
plt.show()"""

"# Elbow\ndef calcular_inercia(X, max_clusters=10):\n    inercia = []\n    for k in range(2, max_clusters):\n        kmeans = KMeans(n_clusters=k, random_state=42)\n        kmeans.fit(X)\n        inercia.append(kmeans.inertia_)\n    return inercia\n\n# Silhouette\ndef calcular_silhouette(X, max_clusters=10):\n    silhouette_scores = []\n    for k in range(2, max_clusters):\n        kmeans = KMeans(n_clusters=k, random_state=42)\n        labels = kmeans.fit_predict(X)\n        score = silhouette_score(X, labels)\n        silhouette_scores.append(score)\n    return silhouette_scores\n\ninercia = calcular_inercia(X_preprocessed)\nsilhouette_scores = calcular_silhouette(X_preprocessed)\n\n# Plotar o gráfico\nplt.figure(figsize=(14, 6))\n\n# Gráfico do método Elbow\nplt.subplot(1, 2, 1)\nplt.plot(range(2, 10), inercia, marker='o', linestyle='--')\nplt.title('Método Elbow para Determinação do Número de Clusters')\nplt.xlabel('Número de Clusters')\nplt.ylabel('Inércia')\nplt.grid()\n\n# Gráfi